<a href="https://colab.research.google.com/github/mequanent/Bioinformatics-hw2/blob/main/hws_110761504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository first using the following command
!npx degit https://github.com/mequanent/Bioinformatics-hw2 -f

npx: installed 1 in 2.254s
> destination directory is not empty. Using --force, continuing
> cloned mequanent/Bioinformatics-hw2#HEAD


**Homework 2**

In [3]:
#pip install Bio   # Please uncomment for every start in colab

In [4]:
import numpy as np
import pandas as pd

#To install the Bio library into my IDLE python editor I used the following two steps.
#    1. cd C:\Users\USER\AppData\Local\Programs\Python\Python39\Scripts
#    2. pip install Bio
#pip install Bio

# Read fasta and parse the seq_names and the sequences
# Reference for parsing the fasta is the original biopython library documentation
# (https://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc11 section 2.4.1)

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test.fasta", "fasta"):
  seq_name.append(seq_record.id)
  #sequence.append(repr(seq_record.seq))
  sequence.append(seq_record.seq)
  #print(len(seq_record))
print(sequence[1])
print(sequence[2])

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  
  return pd.DataFrame(mut, index = columns, columns = columns)

score = "pam250.txt" # score from command line args
score = preprocess_score(score) # score made a data frame to access its values with 
                  # alphabet indices
#print(score)

gop = -10
gex = -2
# A function to determine pairwise alignment score
def pairwise(x, y):
  pair_sum = 0
  x_prev = '' # x_prev and y_prev are used to store immediate previous alphabets
  y_prev = '' # so that extension gaps will be identified
  for i in range(len(x)): # Considering that both x and y have the same length
    x_prev = x_prev == '-' and x[i] == '-' # status of extension gap in the first paired sequence
    y_prev = y_prev == '-' and y[i] == '-' # status of extension gap in second paired sequence
    curr = x[i] == '-' or y[i] == '-' # status of open gap in the paired sequence
    if x_prev or y_prev: # extension gap value will be taken if holds
      pair_sum = pair_sum + gex
    elif curr: # open gap value will be taken if holds
      pair_sum = pair_sum + gop
    else: # score matrix value will be taken otherwise
      pair_sum = pair_sum + score.loc[x[i]][y[i]] 
    x_prev = x[i]
    y_prev = y[i]
  return pair_sum

#pairwise(sequence[0], sequence[1])

# A function to find the total sum of pairs score.
def sumOfPairs(seq):
  sop = 0 # to store sum of pairs value
  for i in range(len(seq)-1): # i iterates as long as there is j in seq as its next sequence
    for j in range(i+1, len(seq)): # j will be the next seq index to be compared with seq[i]
      sop = sop + pairwise(seq[i], seq[j])
  return sop
print(sumOfPairs(sequence))

KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEI--WWARLNDKEGYVPRNLLGLYP
1047


**Homework 3**

In [11]:
import numpy as np
import pandas as pd

from Bio import SeqIO
seq_name = []
sequence = []
# Remember to customize as --input from CLI args
for seq_record in SeqIO.parse("test3.fasta", "fasta"):
  seq_name.append(seq_record.id)
  sequence.append(seq_record.seq)

seq1 = sequence[0]
seq2 = sequence[1]

print("+----- Given Sequences with their respective lengthes ---------+")
print(seq1, len(seq1))
print(seq2, len(seq2))
print("****************************************************************")
gap = -10
score = "pam250.txt" # score from command line args
aln = 'global' # Alignment choice argument from command line interface

TOP = gap
DIAGONAL = 0
LEFT = gap
col = '-' + seq1 # This if for simplicity to avoid repeated use of (len(seq1) + 1)
row = '-' + seq2 # For the same reason

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  return pd.DataFrame(mut, index = columns, columns = columns)

score = preprocess_score(score) # score made a data frame to access its values  
                  # with alphabet indices

# *************************** Global Alignment Section *************************
# Filling score_m and keeping track of the direction for each cell
def globalAlignment(score):
  score_m = np.zeros((len(row), len(col))) # score matrix for global alignment 
  direction_m = np.empty((len(row), len(col)), dtype = object)
  for i in range(len(col)):
    score_m[0][i] = i * gap
  for j in range(len(row)):
    score_m[j][0] = j * gap

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      TOP = score_m[i-1,j] + gap
      DIAGONAL = score_m[i-1][j-1] + score.loc[row[i]][col[j]]
      LEFT = score_m[i,j-1] + gap
      score_m[i][j] = max(TOP, DIAGONAL, LEFT) 
      # ------ Let the direction for each cell be saved as follows 
      if score_m[i][j] == TOP:
        direction_m[i][j] = 'ins'
      elif score_m[i][j] == DIAGONAL:
        direction_m[i][j] = 'sub' 
      else:
        direction_m[i][j] = 'del'

  score_m = score_m.astype(int)
  return score_m, direction_m

# Trace back section for global alignment
def traceBackGlobal():
  score_m, direction_m = globalAlignment(score)
  i = len(seq1)
  j = len(seq2)
  aln1 = ''
  aln2 = ''

  while(not (i == 0 and j == 0)):
    if(direction_m[j][i] == 'sub'):
      aln1 = col[i] + aln1
      aln2 = row[j] + aln2
      i, j = i - 1, j - 1
    elif(direction_m[j][i] == 'del'):
      aln1 = col[i] + aln1
      aln2 = '-' + aln2
      i = i - 1
    else:
      aln1 = '-' + aln1
      aln2 = row[j] + aln2
      j = j - 1
  return aln1, aln2
# End of Trace Back for the Global Alignment
# --------------------------------------------

def showMatched(x, y): # A function to help proper display of the alignments
  between = ''
  for i in range(len(x)):
    if x[i] == y[i]:
      between = between + '|'
    elif x[i] == '-' or y[i] == '-':
      between = between + ' '
    else:
      between = between + '.'
  return between

def displayAlignment(x, y): # A function to display the aligned sequences with their SoP score
  print("+----- You wanted {} alignment and here is the result -----+".format(aln))
  print(x)
  print(showMatched(x, y))
  print(y)
  print("Sum of pairs (SoP) score for the {a} alignment is: {b}".format(a = aln, b = pairwise(x, y)))
# The SoP value verifies the correctness of our alignment
                # if it becomes equal to the right bottom value.
# ************* End of Global Alignment ************************************

# ************* Local Alignment ********************************************
def localAlignment(score):
  score_local = np.zeros((len(row), len(col))).astype(int) # score matrix for local alignment
  direction_l = np.zeros((len(row), len(col)), dtype = object) # to trace direction
  bestScore = 0
  bestScoreCell = (0, 0)

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      TOP = score_local[i-1,j] + gap
      DIAGONAL = score_local[i-1][j-1] + score.loc[row[i]][col[j]]
      LEFT = score_local[i,j-1] + gap
      score_local[i][j] = max(TOP, DIAGONAL, LEFT, 0)

      if score_local[i][j] >= bestScore:
        bestScore = score_local[i][j]
        bestScoreCell = (i, j)
      
      if score_local[i][j] == TOP:
        direction_l[i][j] = 'ins'
      elif score_local[i][j] == DIAGONAL:
        direction_l[i][j] = 'sub' 
      else:
        direction_l[i][j] = 'del'
  return score_local, direction_l, bestScore, bestScoreCell

# Trace back the local alignment
def traceBackLocal():
  score_local, direction_l, bestScore, bestScoreCell = localAlignment(score)
  i = bestScoreCell[0]
  j = bestScoreCell[1]

  al1 = ''
  al2 = ''
  between = ''
  while(score_local[i][j]> 0):
    if(direction_l[i][j] == 'sub'):
      al1 = col[j] + al1
      al2 = row[i] + al2
      i, j = i - 1, j - 1
    elif(direction_l[i][j] == 'del'):
      al1 = col[j] + al1
      al2 = '-' + al2
      j = j - 1
    else:
      al1 = '-' + al1
      al2 = row[i] + al2
      i = i - 1
  return al1, al2

if aln == 'global':
  aln1, aln2 = traceBackGlobal()
  displayAlignment(aln1, aln2) 
elif aln == 'local':
  al1, al2 = traceBackLocal()
  displayAlignment(al1, al2)
else:
  print("Please selected the alignment type.")

+----- Given Sequences with their respective lengthes ---------+
NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGEWCEAQTKNGQGWVPSNYITPVN 57
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP 60
****************************************************************
+----- You wanted global alignment and here is the result -----+
N-LFVALYDFVASGDNTLSITKGEKLRVLGY-NHNG-EWCEAQTKNGQGWVPSNYITPVN
. ...||.|.....|..|....|........ .... ||..|......|.||.|......
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP
Sum of pairs (SoP) score for the global alignment is: 38


In [15]:
# To find all locations of the best score from the local alignment score matrix
score_local, direction_l, bestScore, bestScoreCell = localAlignment(score)
r, c = score_local.shape
bestLoc = []
loc = (0, 0)
for i in range(1, r):
  for j in range(1, c):
    if score_local[i][j] == bestScore:
      loc = (i, j)
      bestLoc.append(loc)
bestLoc

[(56, 53), (57, 54)]

In [ ]:
import Bio.Align as align

aligner = align.PairwiseAligner()
aligner.mode = 'local'
aligner.substitution_matrix = align.substitution_matrices.load("PAM250")
aligner.open_gap_score = -10
alignments = aligner.align(seq1, seq2)
alignments = list(alignments)
print("Number of alignments: %d" % len(alignments))
alignment = alignments[0]
print("Score = %.1f" % alignment.score)
print(alignment)

Number of alignments: 1
Score = 66.0
 NLFVALYDFVASGDNTLSITKGEKLRVLGYNHNGE----WCEAQTKNGQGWVPSNYITPVN
  ...||.|.....|..|....|............|----|--|......|.||.|..
KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWW--ARLNDKEGYVPRNLLGLYP



In [ ]:
print(localAlignment(score)[0])

[[ 0  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  1]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  6 ... 49 44 34]
 [ 0  0  0 ... 39 47 42]
 [ 0  0  0 ... 38 38 47]]
